# Determining Restaurant Inspection Grades
## CS5304 Data Science in the Wild
## Final Project Report

In [ ]:
# Imports
import sys
import argparse
import pyspark
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import lit, when, col
from pyspark.sql.functions import to_date, to_timestamp
from pyspark.sql.types import DateType
from sklearn.model_selection import train_test_split

In [ ]:
finalOutput_df = pd.read_csv(r'FinalOutput.csv') #from Jess Inspection Data
joinedDataset_df = pd.read_csv(r'joinedDataset.csv') #from Jianni PLUTO data
pluto_20v6_df = pd.read_csv(r'pluto_20v6.csv') #from pluto website

## Cleaning Jess's FinalOutput data

In [ ]:
from datetime import datetime as dt

In [ ]:
# convert dates into datetime object
finalOutputTimestampConverted_df = finalOutput_df
finalOutputTimestampConverted_df['__c_8'] = pd.to_datetime(finalOutput_df[['__c_8']].stack(), format='%Y-%m-%d').unstack()

In [ ]:
# only choose times after this date
threshold = dt.strptime("2017-01-01", '%Y-%m-%d')
finalOutputTimestampConverted_df = finalOutputTimestampConverted_df[finalOutputTimestampConverted_df['__c_8'] >= threshold]

In [ ]:
# ensure there are no nulls or duplicates (but should be fine because Jess cleaned ahead of)
finalOutputTimestampConverted_df = finalOutputTimestampConverted_df.dropna()
finalOutputTimestampConverted_df = finalOutputTimestampConverted_df.drop_duplicates()

In [ ]:
# rename row to bbl for merge
finalOutputTimestampConverted_df = finalOutputTimestampConverted_df.rename(columns={'__c_2_4': 'bbl'})

In [ ]:
# # to use if using Jianni's file
# joinedDataset_df = joinedDataset_df.rename(columns = {'Unnamed: 0': 'BBL'})

## Cleaning Pluto Data

In [ ]:
# Select only columns of interest
pluto_20v6_truncated_df = pluto_20v6_df[['bbl', 'yearbuilt', 'lotarea', 'bldgarea', 'comarea', 'resarea', 'officearea', 'retailarea', 'garagearea', 'strgearea', 'factryarea', 'otherarea', 'assessland', 'assesstot']]

In [ ]:
#PLUTO CLEANING
#drop those that were built in year 0 or were null
cleaned_pluto_20v6_truncated_df = pluto_20v6_truncated_df[pluto_20v6_truncated_df.yearbuilt != 0.0]
cleaned_pluto_20v6_truncated_df = cleaned_pluto_20v6_truncated_df[pd.notnull(cleaned_pluto_20v6_truncated_df.yearbuilt)]

# drop those that had no bbl
cleaned_pluto_20v6_truncated_df = cleaned_pluto_20v6_truncated_df[pd.notnull(cleaned_pluto_20v6_truncated_df.bbl)]

# drop those that had 0 or null for lotarea
cleaned_pluto_20v6_truncated_df = cleaned_pluto_20v6_truncated_df[cleaned_pluto_20v6_truncated_df.lotarea != 0.0]
cleaned_pluto_20v6_truncated_df = cleaned_pluto_20v6_truncated_df[pd.notnull(cleaned_pluto_20v6_truncated_df.lotarea)]

# drop those that had 0 or null for bldgarea
cleaned_pluto_20v6_truncated_df = cleaned_pluto_20v6_truncated_df[cleaned_pluto_20v6_truncated_df.bldgarea != 0.0]
cleaned_pluto_20v6_truncated_df = cleaned_pluto_20v6_truncated_df[pd.notnull(cleaned_pluto_20v6_truncated_df.bldgarea)]

# drop any row that has na
cleaned_pluto_20v6_truncated_df = cleaned_pluto_20v6_truncated_df.dropna()

In [ ]:
# convert areas as perectage decimals of overall bldg area
cleaned_pluto_20v6_truncated_df['comareaPCT'] = cleaned_pluto_20v6_truncated_df.comarea / cleaned_pluto_20v6_truncated_df.bldgarea
cleaned_pluto_20v6_truncated_df['resareaPCT'] = cleaned_pluto_20v6_truncated_df.resarea / cleaned_pluto_20v6_truncated_df.bldgarea
cleaned_pluto_20v6_truncated_df['officeareaPCT'] = cleaned_pluto_20v6_truncated_df.officearea / cleaned_pluto_20v6_truncated_df.bldgarea
cleaned_pluto_20v6_truncated_df['retailareaPCT'] = cleaned_pluto_20v6_truncated_df.retailarea / cleaned_pluto_20v6_truncated_df.bldgarea
cleaned_pluto_20v6_truncated_df['garageareaPCT'] = cleaned_pluto_20v6_truncated_df.garagearea / cleaned_pluto_20v6_truncated_df.bldgarea
cleaned_pluto_20v6_truncated_df['strgeareaPCT'] = cleaned_pluto_20v6_truncated_df.strgearea / cleaned_pluto_20v6_truncated_df.bldgarea
cleaned_pluto_20v6_truncated_df['factryareaPCT'] = cleaned_pluto_20v6_truncated_df.factryarea / cleaned_pluto_20v6_truncated_df.bldgarea
cleaned_pluto_20v6_truncated_df['otherareaPCT'] = cleaned_pluto_20v6_truncated_df.otherarea / cleaned_pluto_20v6_truncated_df.bldgarea

cleaned_pluto_20v6_truncated_df = cleaned_pluto_20v6_truncated_df[['bbl', 'yearbuilt', 'lotarea', 'bldgarea', 'comareaPCT', 'resareaPCT', 'officeareaPCT', 'retailareaPCT', 'garageareaPCT', 'strgeareaPCT', 'factryareaPCT', 'otherareaPCT', 'assessland', 'assesstot']]

## Merging the two datasets

In [ ]:
# merge the two dataframes, used inner join because if they dont have a common bbl then we won't be able to match features to inspection grade
inspectionAndPlutoMerged_df = pd.merge(finalOutputTimestampConverted_df, cleaned_pluto_20v6_truncated_df, how='inner', on='bbl')

In [ ]:
print(inspectionAndPlutoMerged_df.columns)

Index(['Unnamed: 0', '_c0', 'level_1', '__c_0', '__c_1', '__c_2', '__c_3',
       '__c_4', '__c_5', '__c_6', '__c_7', '__c_8', '__c_9', '__c_1_0',
       '__c_1_1', '__c_1_2', '__c_1_3', '__c_1_4', '__c_1_5', '__c_1_6',
       '__c_1_7', '__c_1_8', '__c_1_9', '__c_2_0', '__c_2_1', '__c_2_2',
       '__c_2_3', 'bbl', '__c_2_5', 'yearbuilt', 'lotarea', 'bldgarea',
       'comareaPCT', 'resareaPCT', 'officeareaPCT', 'retailareaPCT',
       'garageareaPCT', 'strgeareaPCT', 'factryareaPCT', 'otherareaPCT',
       'assessland', 'assesstot'],
      dtype='object')


In [ ]:
# drop unneccesary columns
inspectionAndPlutoMerged_df = inspectionAndPlutoMerged_df[['__c_0', '__c_1', '__c_2', '__c_3', '__c_4', '__c_5', '__c_7', '__c_8', '__c_9', '__c_1_0', '__c_1_2', '__c_1_3', '__c_1_4', 'bbl', 'yearbuilt', 'lotarea', 'bldgarea', 'comareaPCT', 'resareaPCT', 'officeareaPCT', 'retailareaPCT', 'garageareaPCT', 'strgeareaPCT', 'factryareaPCT', 'otherareaPCT', 'assessland', 'assesstot']]

In [ ]:
# renaming columns
inspectionAndPlutoMerged_df = inspectionAndPlutoMerged_df.rename(columns = {'__c_0' : 'CAMIS',
                                                                            '__c_1' : 'DBA',
                                                                            '__c_2' : 'BORO',
                                                                            '__c_3' : 'BUILDING',
                                                                            '__c_4' : 'STREET',
                                                                            '__c_5' : 'ZIPCODE',
                                                                            '__c_7' : 'CUISINE_DESCRIPTION',
                                                                            '__c_8' : 'INSPECTION_DATE',
                                                                            '__c_9' : 'ACTION',
                                                                            '__c_1_0' : 'VIOLATION_CODE',
                                                                            '__c_1_2' : 'CRITICAL_FLAG',
                                                                            '__c_1_3' : 'SCORE',
                                                                            '__c_1_4' : 'GRADE',
                                                                            })

## Implement Algorithms

In [ ]:
# first we need to cut further columns from the dataframe to only include columns that we consider features / target
forAlgos_df = inspectionAndPlutoMerged_df[['BORO', 'ZIPCODE', 'CUISINE_DESCRIPTION', 'GRADE', 'yearbuilt', 'lotarea', 'bldgarea', 'comareaPCT', 'resareaPCT', 'officeareaPCT', 'retailareaPCT', 'garageareaPCT', 'strgeareaPCT', 'factryareaPCT', 'otherareaPCT', 'assessland', 'assesstot']]

In [ ]:
# split data up, X are features while Y are targets
X = forAlgos_df[['BORO', 'ZIPCODE', 'CUISINE_DESCRIPTION', 'yearbuilt', 'lotarea', 'bldgarea', 'comareaPCT', 'resareaPCT', 'officeareaPCT', 'retailareaPCT', 'garageareaPCT', 'strgeareaPCT', 'factryareaPCT', 'otherareaPCT', 'assessland', 'assesstot']]
Y = forAlgos_df[['GRADE']]

In [ ]:
inspectionAndPlutoMerged_df.to_csv(r'/Users/TobyLeung/CS5304/Group Project/inspectionAndPlutoMerged.csv', index='false')
forAlgos_df.to_csv(r'/Users/TobyLeung/CS5304/Group Project/forAlgos.csv', index='false')

In [ ]:
# split data with train-test split
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=0.2)

In [ ]:
cusineCounts = forAlgos_df['CUISINE_DESCRIPTION'].value_counts()
print(cusineCounts)

American             14759
Chinese               6602
Café/Coffee/Tea       4776
Pizza                 3206
Mexican               2744
                     ...  
Fruits/Vegetables       14
Iranian                 12
Czech                   12
Scandinavian            11
Southwestern             6
Name: CUISINE_DESCRIPTION, Length: 82, dtype: int64


In [ ]:
# fit linearRegression Model
from sklearn.linear_model import LinearRegression
clf = LinearRegression()

clf.fit(x_train, y_train)

ValueError: could not convert string to float: 'Queens'